In [2]:
# -*- coding: utf-8 -*-
"""
Exemple simple de MNIST avec PyTorch
Exemple de nn.Module avec deux couches denses linéaires
Boucle d'apprentissage simple
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn.functional as F
fonction_cout = F.cross_entropy

from torch import nn
# Définition de l'architecture du RNA
class RNASimple(nn.Module):
    def __init__(self):
        super().__init__()
        self.couche_lineaire1 = nn.Linear(784, 30)
        self.couche_lineaire2 = nn.Linear(30, 10)
    def forward(self, lot_X):
            lot_X = F.relu(self.couche_lineaire1(lot_X))
            return self.couche_lineaire2(lot_X)
modele = RNASimple()
    
from torch import optim
optimiseur = optim.SGD(modele.parameters(), lr=0.05)

# Chargement des données de MNIST
import pickle, gzip
fichier_donnees = gzip.open(r"mnist.pkl.gz", 'rb')
((donnees_ent_X, donnees_ent_Y),(donnees_valid_X, donnees_valid_Y),(donnees_test_X,donnees_test_Y)) = pickle.load(fichier_donnees, encoding="latin-1")
fichier_donnees.close()

# Conversion des données en type toch.Tensor
donnees_ent_X, donnees_ent_Y, donnees_test_X,donnees_test_Y = map(torch.tensor, (donnees_ent_X, donnees_ent_Y, donnees_test_X,donnees_test_Y))

# Création des objets DataLoader pour itérer par lot
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
ds_ent = TensorDataset(donnees_ent_X, donnees_ent_Y)
dl_ent = DataLoader(ds_ent, batch_size=100, shuffle=True)

nb_epochs = 10
# Boucle d'apprentissage
for epoch in range(nb_epochs):
    cout_total_ent = 0 
    
    # Boucle d'apprentissage par mini-lot pour une epoch
    for lot_X, lot_Y in dl_ent:
        optimiseur.zero_grad() # Remettre les dérivées à zéro
        lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
        cout = fonction_cout(lot_Y_predictions, lot_Y)
        cout_total_ent +=cout
        cout.backward() # Calcul des gradiants par rétropropagation
        optimiseur.step() # Mise à jour des paramètres
        
    cout_moyen_ent = cout_total_ent/len(dl_ent)
    print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')

-------- > epoch 1:  coût moyen entraînement = 0.7796586751937866
-------- > epoch 2:  coût moyen entraînement = 0.35505351424217224
-------- > epoch 3:  coût moyen entraînement = 0.3114163279533386
-------- > epoch 4:  coût moyen entraînement = 0.28549620509147644
-------- > epoch 5:  coût moyen entraînement = 0.2659691870212555
-------- > epoch 6:  coût moyen entraînement = 0.2493172287940979
-------- > epoch 7:  coût moyen entraînement = 0.2349935621023178
-------- > epoch 8:  coût moyen entraînement = 0.22249244153499603
-------- > epoch 9:  coût moyen entraînement = 0.21091149747371674
-------- > epoch 10:  coût moyen entraînement = 0.20132428407669067


In [2]:
print(modele)

RNASimple(
  (couche_lineaire1): Linear(in_features=784, out_features=30, bias=True)
  (couche_lineaire2): Linear(in_features=30, out_features=10, bias=True)
)


In [3]:
for parametre in modele.parameters():
    print(parametre.shape)

torch.Size([30, 784])
torch.Size([30])
torch.Size([10, 30])
torch.Size([10])
